In [11]:
import tensorflow as tf

In [12]:
# 创建变量
# tf.random_normal 方法返回形状为(1，4)的张量，并命名为W
#   它的4个元素符合均值为100、标准差为0.35的正态分布。
# 
W = tf.Variable(initial_value=tf.random_normal(shape=(1, 4), mean=100, stddev=0.35), name="W")
b = tf.Variable(tf.zeros([4]), name="b")

In [13]:
# 已经有名字W，b了，不再是variable0，variable1
[W, b]

[<tf.Variable 'W_1:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'b_1:0' shape=(4,) dtype=float32_ref>]

In [14]:
# 初始化变量
# 创建会话（之后小节介绍，现在理解为运行环境，从会话可以得到资源）
sess = tf.Session()
# 使用 global_variables_initializer 方法初始化全局变量 W 和 b
# 执行操作、给W和b赋值
sess.run(tf.global_variables_initializer())
# 执行操作、来看结果
# W：正太分布初始化（tf.random_normal)
# b: 零值（tf.zero)
sess.run([W, b])

[array([[100.58703,  99.85105,  99.76723, 100.18915]], dtype=float32),
 array([0., 0., 0., 0.], dtype=float32)]

In [15]:
# 执行更新变量 b 的操作
# 用 assign 或者 assign_add，传入一个一阶张量，做向量相加
# 执行操作、更新 b
sess.run(tf.assign_add(b, [1, 1, 1, 1]))

array([1., 1., 1., 1.], dtype=float32)

In [16]:
# 执行操作、查看变量 b 是否更新成功
sess.run(b)

array([1., 1., 1., 1.], dtype=float32)

### Saver 使用示例：保存和恢复变量
```PYTHON
# 定义要保存的变量
# v1是变量，name='v1'是定义变量操作的名称
v1 = tf.Variable(..., name='v1')  
v2 = tf.Variable(..., name='v2')
# 指定需要保存和恢复的变量
# saver接受字典、变量列表两种方式的变量
#  字典：{'v1':v1, ...}  
#    'v1': 保存到文件中的变量名
#     v1 : 数据流图中的变量
#  变量列表：[v1, ...]
#     v1 : 数据流图中变量v1的操作name=‘v1’中的'v1'作为文件中的变量名
#  与上面的变量列表等价：
#     更显示的方式保存，v.op.name就是上面操作的名称（如name='v1'中的‘v1’）
saver = tf.train.Saver({'v1': v1, 'v2': v2})
saver = tf.train.Saver([v1, v2])
saver = tf.train.Saver({v.op.name: v for v in [v1, v2]})
# 执行变量保存操作
#  sess：数据流图相关信息在这里
#  ‘my-model’: 文件名前缀
#  global_step: 推荐使用，指明了当前训练的步数，回溯/查看问题/找步骤中哪步的某个训练指标最好等
tf.train.saver.save(sess, 'my-model', global_step=0) # ==> filename: 'my-model-0'
```

In [17]:
# 创建Saver
saver = tf.train.Saver({'W': W, 'b': b})
# 存储变量到文件 './summary/test.ckpt-0'
saver.save(sess, './summary/test.ckpt', global_step=0)
# 打开目录查看
#   test.ckpt-0.data-00000-of-00001: 变量实际值
#   test.ckpt-0.meta: 数据流图的结构
#   test.ckpt-0.index: 索引、数据流图很大时（迭代步骤的增加），用索引可以快速找回数据
# 但是这些后缀都已经被tf封装好了，我们只需要关注前缀‘./summary/test.ckpt-0’就够了

'./summary/test.ckpt-0'

In [18]:
# 再次执行更新变量 b 的操作，为了之后与从文件中恢复的数据对比
sess.run(tf.assign_add(b, [1, 1, 1, 1]))
# 获取变量 b 的最新值
sess.run(b)

array([2., 2., 2., 2.], dtype=float32)

In [19]:
# 从文件中恢复变量 b 的值
saver.restore(sess, './summary/test.ckpt-0')
# 查看变量 b 是否恢复成功
sess.run(b)

array([1., 1., 1., 1.], dtype=float32)

In [10]:
# 上面只是恢复数据，实际训练中，往往恢复数据不够用，还需要恢复整个数据流图的结构
# 从文件中恢复数据流图结构（即上面对应的*.meta文件）
# tf.train.import_meta_graph